In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                console.log("Double tap.");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>
<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The source code for this IPython notebook is hidden.
To view the source code, <a href="javascript:code_toggle()">click here</a>.

# Structured Summary (Article Annotation)
Use this form every time you read a research article to create notes that become your "zeroth" draft for writing. Paraphrase in your own words. If you copy a quote, surround the phrase in your notes with quotation marks.

In [2]:
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox
from IPython.display import display, Markdown, Latex, HTML
from IPython.display import clear_output
import os.path
import math
import copy
import csv
import datetime
import time
output = widgets.Output()

question_fields = [
    widgets.Text(placeholder='First', disabled=False),#First author
    widgets.Text(placeholder='Second', disabled=False),#Second author
    widgets.Text(placeholder='Third', disabled=False),#Third author
    widgets.Text(placeholder='Year', disabled=False),#Year
    widgets.Text(placeholder='Title', disabled=False),#URL,
    widgets.Text(placeholder='https://ui.adsabs.harvard.edu/abs/2019A%26A...627A.165H/abstract', disabled=False),#Title
    widgets.Textarea(placeholder='Answer', disabled=False),#C
    widgets.Textarea(placeholder='Answer', disabled=False),#E
    widgets.Textarea(placeholder='Answer', disabled=False),#R
    widgets.Textarea(placeholder='Answer', disabled=False),#I
    widgets.Text(placeholder='Answer', disabled=False),#Credibility
    widgets.Text(placeholder='Answer', disabled=False),#Purpose of study
    widgets.Textarea(placeholder='Answer', disabled=False),#Questions about study
    widgets.Textarea(placeholder='Answer', disabled=False),#How does the study connect to my research?
]

questions_short = ['First author',#These are the names of the columns in the csv.
                   'Second author',
                   'Third author',
                   'Year',
                   'Title',
                   'URL',
                   'Claim',
                   'Evidence',
                   'Reasoning',
                   'Implications',
                   'Credibility',
                   'Purpose of study',
                   'Questions about study',
                   'How does it connect?',
                  ]

#This is the text that comes with the questions. The first 3 (authors) are grouped horizontally, so they only require 1 line.
#The last two (Type and Keywords) are done explicitly, below, after the following question index:
type_and_keyword_index = 9
redstar = ' __<font color=red>*</font>__'
questions_full = ['Authors'+redstar,
                  'Year'+redstar,
                  'Title'+redstar,
                  'Link / url',
                  'Claim (i.e., hypothesis found in Title, Abstract, Intro, etc)',
                  'Evidence (i.e., data, measurements, outcomes found in Results)',
                  'Reasoning (i.e., connect evidence to claim found in Interpretation or Discussion)',
                  'Implications (i.e., What next? Found in Interpretation or Discussion)',
                  'Credibility of author (i.e., affiliations, prior publications)',
                  'Purpose of the study',
                  'Questions about the study?',
                  'How does the study connect to my research question?',   
]









#Now starts the main code.
#First we test that the number of question texts, question fields, and csv columns are consistent.
questions_short.append('Type')
questions_short.append('Keywords')
if len(question_fields) != len(questions_short)-2:
    print(len(question_fields),len(questions_short)-2)
    print('ERROR: Wrong number of questions fields vs csv columns.')
if len(questions_full) != len(questions_short)-4:
    print('ERROR: Wrong number of questions (full text) vs question fields.')
    print('Length questions_full: '+str(len(questions_full)))
    print('Length questions_short: '+str(len(questions_short)-4))
    
    
    
    
#    
kw_path = 'keywords.txt'#Path to textfile that contains a keyword on each line.
#THIS HANDLES THE BIG GRID OF CHECKBOXES FOR PAPER KEYWORDS:
def create_keyword_matrix():
    ncol_max = 3  #4 columns
    nrow_min = 4  #start with 4 rows minimum before populating the other columns.
    
    if os.path.isfile(kw_path):
        keywords = open(kw_path,'r').read().splitlines()
        keywords.sort()
    else:
        keywords = ['Exoplanets']
    N_keywords = len(keywords)
    nrow = max([nrow_min,math.ceil(N_keywords/ncol_max)])#Based on the number of keywords, determine the number of rows and columns
    ncol = min([ncol_max,math.ceil(N_keywords/nrow)])
    M=[[]]#This will be built up with the grid.
    col = 0#Start with column zero.
    for i,kw in enumerate(keywords):
        if (i % nrow) == 0 and i != 0:
            M.append([])
            col+=1 
        M[col].append(widgets.Checkbox(value=False,description=kw,disabled=False))
    VBoxes = []
    for col in M:
        VBoxes.append(VBox(col))
    return(HBox(VBoxes))
keyword_matrix = create_keyword_matrix()
keyword_other = widgets.Text(placeholder='Geophysics, Compact objects, Galaxies')


# ==========================================================================================
def append_options(path,items):
        with open(path,'a+') as f:
            for i in items:
                f.write(i+'\n')

# THIS HANDLES THE PAPER TYPES RADIO-BUTTONS:
tp_path = 'types.txt'
def read_types(tp_path):
    if os.path.isfile(tp_path):
        types = open(tp_path,'r').read().splitlines()
        types.sort()
    else:
        types = ['Observational', 'Theoretical']
        append_options(tp_path,types)
    types_options=copy.deepcopy(types)
    types_options.append('Other:')#Copy it to allow Other: to be added, without being a proper keyword.
    return(types_options)
types_options=read_types(tp_path)
        

type_radio = widgets.RadioButtons(options=types_options,description='',disabled=False)
type_other = widgets.Text(placeholder='Answer', disabled=True)
def disabler(event):
    if type_radio.value == 'Other:':
        type_other.set_trait('disabled',False)
    else:
        type_other.set_trait('disabled',True)

# =============================================

def button_enabler(event):
    if question_fields[0].value != '' and question_fields[3].value != '' and question_fields[4].value != '':
        submit_button.set_trait('disabled',False) 
        submit_button.set_trait('button_style','success')
    else:
        submit_button.set_trait('disabled',True)
        submit_button.set_trait('button_style','danger')
        
        
# THIS HANDLES THE SUBMIT-BUTTON: 
#EVERYTHING HERE NEEDS TO BE COPIES. ARRAYS FROM ABOVE MAY NOT BE APPENDED.
def submit(event):
    #First add the new keywords and types to their respective files.
    if type_radio.value == 'Other:':#If an other was set, we need to first select it to be the output, rather than the state of the radio button.
        papertype = type_other.value
        types_options = read_types(tp_path)
        if (papertype in types_options) == False:#And if its not already in types.txt, we need to add it. (This basically catches the event that the user puts an existing value as Other.)
            append_options(tp_path,[papertype])#Write to file.
    else:
        papertype = copy.deepcopy(type_radio.value)
        
    if len(keyword_other.value) != 0:
        new_keywords=[]#Redefined each time submit button is pressed.
        for nkw in keyword_other.value.split(','):#Split the new keyword string on the commas and remove trailing and leading spaces for each element.
            new_keywords.append(nkw.lstrip().rstrip())
        append_options(kw_path,new_keywords)#Write to file.
        
    #Add all the output values to a single list.    
    outlist = [i.value for i in question_fields]#Redefined each time button is pressed...
    outlist.append(papertype)#...so this is allowed.
    

    #Add the selected keywords as a list to this list:
    selected_keywords = ''#Redefined each time button is pressed.
    global keyword_matrix
    for i in keyword_matrix.children:
        for j in i.children:
            if j.value == True:
                selected_keywords+=j.description+' '
    outlist.append(selected_keywords)
    outlist.append(str(datetime.datetime.now()))
    csv_columns = copy.deepcopy(questions_short)
    csv_columns.append('Date added')
    
    if os.path.isfile('references.csv') != True:#Create new references file if it doesnt already exist.
        print('Creating new references.csv.')
        with open('references.csv', 'a+') as f:
            writer = csv.writer(f)
            writer.writerow(csv_columns)

    with open('references.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(outlist)
    
    #Finally reset the entire form:
    clear_output()
    for qf in question_fields:
        qf.value = ''
    for i in keyword_matrix.children:
        for j in i.children:
            j.value = False
    keyword_other.value = ''
    type_other.value = ''
    types_options = read_types(tp_path)
    type_radio.options = types_options
    keyword_matrix = create_keyword_matrix()
    spawn_form()
    
    
    

submit_button = widgets.Button(value=False,description='Submit',disabled=True,button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                     tooltip='Submit answers',icon='check')
submit_button.on_click(submit)
type_radio.observe(disabler)
question_fields[0].observe(button_enabler)
question_fields[3].observe(button_enabler)
question_fields[4].observe(button_enabler)


        


def spawn_form():
    display(Markdown(questions_full[0]))
    display(HBox([question_fields[0],question_fields[1],question_fields[2]]))
    for i,q in enumerate(question_fields):
        if i >= 3:
            display(Markdown(questions_full[i-2]))
            display(q)
        if i == type_and_keyword_index:
            display(Markdown('What is the over-arching type of study (e.g. observational or theoretical)?'))
            display(type_radio)
            display(type_other)
            display(Markdown('Please assign keywords that describe the topic of the paper.'))
            display(keyword_matrix)
            display(Markdown('Other keywords (separate multiple with commas)'))
            display(keyword_other)
    display(submit_button)
spawn_form()


Authors __<font color=red>*</font>__

Year __<font color=red>*</font>__

Text(value='', placeholder='Year')

Title __<font color=red>*</font>__

Text(value='', placeholder='Title')

...

Questions about the study?

Textarea(value='', placeholder='Answer')

How does the study connect to my research question?

Textarea(value='', placeholder='Answer')

Button(button_style='danger', description='Submit', disabled=True, icon='check', style=ButtonStyle(), tooltip=…